In [ ]:
import numpy as np
from scipy.optimize import minimize

class Child:
    def __init__(self, age, height_for_age, mortality):
        self.age = age
        self.height_for_age = np.random.normal(-2, 1) 
        self.mortality = mortality

class Population:
    def __init__(self, num_children):
        self.children = [Child(np.random.uniform(0, 5), 0, np.random.uniform(0, 0.1)) for _ in range(num_children)]

class Intervention:
    def __init__(self, name, cost, effectiveness, effect_on_height, effect_on_mortality):
        self.name = name
        self.cost = cost
        self.effectiveness = effectiveness
        self.effect_on_height = effect_on_height
        self.effect_on_mortality = effect_on_mortality

class OptimaModel:
    def __init__(self, population, interventions, budget):
        self.population = population
        self.interventions = interventions
        self.budget = budget

    def objective(self, coverage):
        total_cost = sum(intervention.cost * cov for intervention, cov in zip(self.interventions, coverage))

        for child in self.population.children:
            child.height_for_age = np.random.normal(-2, 1)

        for intervention, cov in zip(self.interventions, coverage):
            for child in self.population.children:
                if np.random.rand() < intervention.effectiveness * cov:
                    child.height_for_age += intervention.effect_on_height
                    child.mortality += intervention.effect_on_mortality

        total_mortality = sum(child.mortality for child in self.population.children)
        total_stunting = sum(1 for child in self.population.children if child.height_for_age < -2)

        return total_cost + total_mortality + total_stunting

    def optimize(self):
        initial_guess = np.random.uniform(0, 1, len(self.interventions)) 
        initial_guess /= np.sum(initial_guess)
        bounds = [(0, 1)] * len(self.interventions)
        constraints = {'type': 'eq', 'fun': lambda x: sum(x) - 1}
        result = minimize(self.objective, initial_guess, bounds=bounds, constraints=constraints, method='SLSQP')
        return result


AMS = Intervention("Antenatal micronutrient supplementation", cost=50, effectiveness=0.6, effect_on_height=0.2, effect_on_mortality=-0.1)
PEPS = Intervention("Protein-energy supplementation", cost=100, effectiveness=0.5, effect_on_height=0.1, effect_on_mortality=-0.05)
Breastfeeding = Intervention("Breastfeeding promotion", cost=70, effectiveness=0.7, effect_on_height=0.3, effect_on_mortality=-0.2)
IYCF = Intervention("Infant and young child feeding program", cost=80, effectiveness=0.4, effect_on_height=0.1, effect_on_mortality=-0.1)
PPCF = Intervention("Public provision of complementary foods", cost=90, effectiveness=0.5, effect_on_height=0.2, effect_on_mortality=-0.15)
VitaminA = Intervention("Vitamin A supplementation", cost=30, effectiveness=0.6, effect_on_height=0.1, effect_on_mortality=-0.1)

population = Population(1000)  


budget = 100000  
model = OptimaModel(population, [AMS, PEPS, Breastfeeding, IYCF, PPCF, VitaminA], budget)


result = model.optimize()


for i, intervention in enumerate(model.interventions):
    for child in model.population.children:
        child.height_for_age += intervention.effect_on_height * result.x[i]
        child.mortality += intervention.effect_on_mortality * result.x[i]

for i, intervention in enumerate(model.interventions):
    print(f"Optimal coverage for {intervention.name}: {result.x[i]}")

total_mortality = sum(child.mortality for child in model.population.children)
total_stunting = sum(1 for child in model.population.children if child.height_for_age < -2)

print(f"Total mortality: {total_mortality}")
print(f"Total number of stunted children: {total_stunting}")
